# Luke Formulary Attempt
### v2.0

Let's start simple and see if we can determine which drugs are in a payer's formulary (reject code for formularies is 70)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('../data/dim_claims.csv')


In [3]:
print(df.shape)
list(df.columns)
df.head(5)

(1335576, 5)


,dim_claim_id,bin,drug,reject_code,pharmacy_claim_approved
0,1,417380,A,75.0,0
1,2,999001,A,NaN,1
2,3,417740,A,76.0,0
3,4,999001,A,NaN,1
4,5,417740,A,NaN,1


In [4]:
#X_train,X_test,y_train,y_test = train_test_split(X,y,
#                                                 test_size = .25,
#                                                 random_state = 614,
#                                                 shuffle = True,
#                                                 stratify = y)

#Start with 25% of the data set to keep the size managable and avoid biasing our results
df_train = df.sample(frac = .75)

#For each payer, why is the drug rejected?
payers = df_train['bin'].unique()
drugs = df_train['drug'].unique()
print(drugs)
print(payers)
print(type(payers))





['C' 'B' 'A']
[417740 999001 417614 417380]
<class 'numpy.ndarray'>


In [5]:
non_form_train = df_train[df_train['reject_code']==70]

In [6]:
non_form_train.shape

(189320, 5)

In [7]:
#cl = ['Payer']
#cl.extend(drugs)
#print(type(cl))

#print(cl)
df_rej= pd.DataFrame(columns=drugs)
df_rej.insert(loc=0,column='Payer',value=payers)

#den = df_train[(df_train['bin'] == payers[0])  & (df_train['drug']==drugs[0])].shape[0]
#rej = df_train[(df_train['bin'] == payers[0])  & (df_train['drug']==drugs[0]) & (df_train['pharmacy_claim_approved']==0)].shape[0]
#print(den)
#print(rej)
i = 0

#df_rej['Payer'] = payers

for payer in payers:
    for drug in drugs:
        den = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug)].shape[0]
        #print("Den = ",den)
        rej = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug) & (df_train['pharmacy_claim_approved']==0)].shape[0]
        acc = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug) & (df_train['pharmacy_claim_approved']==1)].shape[0]
        rejfrac = rej/den
     #   print("Payer ",payer,"rejects drug",drug," = ",rejfrac)
        df_rej.loc[df_rej['Payer'] == payer, [drug]] = rejfrac
        print("Total = ",acc,"+",rej," = ", acc+rej,", should equal", den)
        
df_rej.head()     


#df_train[(df_train['bin'] == 417740)  & (df_train['drug']=='B') & (df_train['pharmacy_claim_approved']==1)]


Total =  0 + 37696  =  37696 , should equal 37696
Total =  0 + 41332  =  41332 , should equal 41332
Total =  73457 + 8222  =  81679 , should equal 81679
Total =  101412 + 11447  =  112859 , should equal 112859
Total =  111176 + 12365  =  123541 , should equal 123541
Total =  219428 + 24366  =  243794 , should equal 243794
Total =  48801 + 5318  =  54119 , should equal 54119
Total =  0 + 59051  =  59051 , should equal 59051
Total =  0 + 117408  =  117408 , should equal 117408
Total =  0 + 30580  =  30580 , should equal 30580
Total =  29871 + 3352  =  33223 , should equal 33223
Total =  0 + 66400  =  66400 , should equal 66400


,Payer,C,B,A
0,417740,1.000000,1.000000,0.100662
1,999001,0.101427,0.100088,0.099945
2,417614,0.098265,1.000000,1.000000
3,417380,1.000000,0.100894,1.000000


So, using this simple method, we se that 

Payer 999001 has all three drugs in their formulary


Payer 417614 has only C in their formulary

Payer 417740 has only A in their formulary

Payer 417380 has has only B in their formlary

If this is correct, then we should see only 70 rejection codes for non-forumlary drugs and non-70 codes for formulary rejections

In [10]:
df_code= pd.DataFrame(columns=drugs)

df_code.insert(loc=0,column='Payer',value=0)
df_code.insert(loc=1,column='Acc/Code',value=0)

codes = df_train['reject_code'].unique()
print(codes)
print(df_train[df_train['pharmacy_claim_approved']==1]['reject_code'].unique())
print(df_train[df_train['pharmacy_claim_approved']==0]['reject_code'].unique())
#So, all codes seem to be what they should be
df_train['reject_code'] = df['reject_code'].fillna(-1)

print(df_train.head(10))

rep = len(codes)
print(rep)

p = payers.repeat(rep)
print(p)

df_code['Payer'] = p
 
#pd.DataFrame([[payer for payer in payers],[1]], columns=['Payer','Acc/Code'])
for payer in payers:
    df_code.loc[df_code['Payer'] == payer,['Acc/Code']] = codes

df_code.head(20)

[75. -1. 70. 76.]
[-1.]
[75. 70. 76.]
         dim_claim_id     bin drug  reject_code  pharmacy_claim_approved
200046         200047  417740    C         75.0                        0
777097         777098  999001    B         -1.0                        1
45489           45490  999001    C         -1.0                        1
1129735       1129736  999001    A         -1.0                        1
641774         641775  417740    A         -1.0                        1
144755         144756  999001    A         -1.0                        1
590379         590380  417614    A         70.0                        0
119412         119413  999001    C         -1.0                        1
77965           77966  417740    C         75.0                        0
1003339       1003340  417614    A         70.0                        0
4
[417740 417740 417740 417740 999001 999001 999001 999001 417614 417614
 417614 417614 417380 417380 417380 417380]


,Payer,Acc/Code,C,B,A
0,417740,75.0,NaN,NaN,NaN
1,417740,-1.0,NaN,NaN,NaN
2,417740,70.0,NaN,NaN,NaN
3,417740,76.0,NaN,NaN,NaN
4,999001,75.0,NaN,NaN,NaN
5,999001,-1.0,NaN,NaN,NaN
6,999001,70.0,NaN,NaN,NaN
7,999001,76.0,NaN,NaN,NaN
8,417614,75.0,NaN,NaN,NaN
9,417614,-1.0,NaN,NaN,NaN


In [11]:


#den = df_train[(df_train['bin'] == payers[0])  & (df_train['drug']==drugs[0])].shape[0]
#rej = df_train[(df_train['bin'] == payers[0])  & (df_train['drug']==drugs[0]) & (df_train['pharmacy_claim_approved']==0)].shape[0]
#print(den)
#print(rej)
i = 0

#df_code['Payer'] = payers

for payer in payers:
    for drug in drugs:
        for code in codes:
            den = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug)].shape[0]
        #print("Den = ",den)
            if(code>0):
                rej = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug) 
                               & (df_train['pharmacy_claim_approved']==0) & (df_train['reject_code']==code)].shape[0]
            else:
                rej = 0    
            acc = df_train[(df_train['bin'] == payer)  & (df_train['drug']==drug) & (df_train['pharmacy_claim_approved']==1)].shape[0]
            rejfrac = rej/den
            accfrac = acc/den
     #   print("Payer ",payer,"rejects drug",drug," = ",rejfrac)
            if(code>0):
                df_code.loc[(df_code['Payer'] == payer) & (df_code['Acc/Code']==code), [drug]] = rejfrac
            else:
                df_code.loc[(df_code['Payer'] == payer) & (df_code['Acc/Code']==code), [drug]] = accfrac
        #print("Total = ",acc,"+",rej," = ", acc+rej,", should equal", den)
        
df_code.head(25)     

,Payer,Acc/Code,C,B,A
0,417740,75.0,1.000000,0.000000,0.000000
1,417740,-1.0,0.000000,0.000000,0.899338
2,417740,70.0,0.000000,1.000000,0.000000
3,417740,76.0,0.000000,0.000000,0.100662
4,999001,75.0,0.000000,0.000000,0.000000
5,999001,-1.0,0.898573,0.899912,0.900055
6,999001,70.0,0.000000,0.000000,0.000000
7,999001,76.0,0.101427,0.100088,0.099945
8,417614,75.0,0.000000,1.000000,0.000000
9,417614,-1.0,0.901735,0.000000,0.000000


A more detailed view now emerges

999001: No drug is rejeced due to formulay or requires prior authorization. All druges are rejected due to code 76

417614: Drug A is not in formulary (always rejected, always due to code 70)
        Drug B is on formulary but does not have preferred status and requires a prior authorization (is always rejected and rejected due to code 75)
        Drug C is accepted most of the time and when rejected it is due to code 76 (that the plan limitations have been exceeded)
        
417740: Drug A is accepted most of the time and when rejected it is due to code 76 (that the plan limitations have been exceeded)
        Drug B is not in formulary (always rejected, always due to code 70)
        Drug C is on formulary but does not have preferred status and requires a prior authorization (is always rejected and rejected due to code 75)
        
417380: Drug A is on formulary but does not have preferred status and requires a prior authorization (is always rejected and rejected due to code 75)
        Drug B is accepted most of the time and when rejected it is due to code 76 (that the plan limitations have been exceeded)
        Drug C is not in formulary (always rejected, always due to code 70)

## Looking at Individual Payer 417614

In [18]:
dfpa = pd.read_csv('../data/dim_pa.csv')
bridge = pd.read_csv('../data/bridge.csv')
dfpa.head(10)


,dim_pa_id,correct_diagnosis,tried_and_failed,contraindication,pa_approved
0,1,1,1,0,1
1,2,1,0,0,1
2,3,0,0,1,1
3,4,1,1,0,1
4,5,0,1,0,1
5,6,1,1,0,1
6,7,0,1,0,1
7,8,1,0,1,0
8,9,1,0,1,0
9,10,1,1,0,1


In [19]:
bridge.head(10)

,dim_claim_id,dim_pa_id,dim_date_id
0,1,1.0,1
1,2,NaN,1
2,3,2.0,1
3,4,NaN,1
4,5,NaN,1
5,6,NaN,1
6,7,NaN,1
7,8,NaN,1
8,9,NaN,1
9,10,3.0,1
